In [2]:
# Execute if necessary
# %%capture
#!pip install numpy seaborn matplotlib pandas openml

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List
import openml
import sklearn

# Práctica 4: Overfitting

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado se llama LDPA y puede leer su descripción en la siguiente liga

https://www.openml.org/d/1483

In [4]:
# Descarga la metadata del dataset
dataset_info = openml.datasets.get_dataset(1483, download_data=False)

# Obtiene el nombre de la columna a predecir
target = dataset_info.default_target_attribute

(
    features, # Dataframe con las características que se pueden utilizar para predecir
    outputs, # Columna a predecir
    categorical_mask, # Máscara que indica que columnas de todas las características son categoricas
    columns # Lista con el nombre de las características
)= dataset_info.get_data(
    dataset_format="dataframe", target=target
)

categorical_mask = np.array(categorical_mask)
columns = np.array(columns)

In [5]:
print(f"La columna a predecir se llama '{target}'")
print(f"Todas las características son {str(columns)}")
print(f"Las características categóricas son {str(columns[categorical_mask])}")
print(f"Las características numéricas son {str(columns[~categorical_mask])}")

La columna a predecir se llama 'Class'
Todas las características son ['V1' 'V2' 'V3' 'V4' 'V5' 'V6' 'V7']
Las características categóricas son ['V1' 'V2']
Las características numéricas son ['V3' 'V4' 'V5' 'V6' 'V7']


In [6]:
# Impresión de las características
features

V1 V2        V3        V4        V5        V6        V7
0       1  1  105794.0  105777.0  148933.0  108479.0  109365.0
1       1  4  105795.0  105778.0  155325.0   96952.0  161363.0
2       1  3  105796.0  105779.0  156652.0  101610.0  148870.0
3       1  1  105797.0  105780.0  149736.0  107135.0  102479.0
4       1  2  105798.0  105781.0  155978.0  125455.0  106203.0
...    .. ..       ...       ...       ...       ...       ...
164855  5  2   43687.0   43679.0  106611.0  123166.0   12432.0
164856  5  1   43688.0   43680.0  120821.0  119848.0   87463.0
164857  5  4   43689.0   43681.0  104999.0  110823.0  161536.0
164858  5  2   43690.0   43682.0  102191.0  112520.0   10811.0
164859  5  1   43691.0   43683.0  106656.0  113422.0   80572.0

[164860 rows x 7 columns]

In [7]:
outputs

0         3
1         3
2         3
3         3
4         3
         ..
164855    3
164856    3
164857    3
164858    3
164859    3
Name: Class, Length: 164860, dtype: category
Categories (11, object): ['1' < '2' < '3' < '4' ... '8' < '9' < '10' < '11']

A continuación se realiza la partición de __train__ y __test__. __No debe utilizar la partición de test por ningún motivo__.

In [8]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    features, 
    outputs, 
    test_size=0.5, 
    random_state=11
)

## Asignación 1 &#x1F921;

Utilizando las técnicas vistas en clase, realicé su mejor esfuerzo para obtener el mejor modelo lineal posible utilizando __únicamente el conjunto de entrenamiento__ (X_train y y_train). Puede utilizar bibliotecas, pero únicamente las técnicas vistas en clase y debe ir generando métricas o visualizaciones que respalden su toma de decisiones. Debe tener e imprimir una estimación de $E_{out}$ utilizando su conjunto de entrenamiento. La métrica utilizada será la entropía cruzada.

- La columna a predecir se llama 'Class'
- Todas las características son ['V1' 'V2' 'V3' 'V4' 'V5' 'V6' 'V7']
- Las características categóricas son ['V1' 'V2']
- Las características numéricas son ['V3' 'V4' 'V5' 'V6' 'V7']

1. Como ya estan convertidos los datos categoricos no habrá necesidad de convertirlos
2. Elegí realizar Cross Validation porque puedes evaluar el rendimiento de tu modelo de manera más robusta que utilizando simplemente un conjunto de entrenamiento y prueba. También te permite comparar el rendimiento de diferentes modelos y seleccionar el mejor modelo para tus datos.

In [11]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

En este caso, el modelo que se va a utilizar es una regresión lineal

In [17]:
model = LinearRegression()
#Se utiliza el cross validation con 100 iteraciones y se obtiene el score de cada iteración
kf = KFold(n_splits=100, shuffle=True, random_state=42)
#Se utiliza el score R2 para evaluar el modelo
scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
print("Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation") 
print("R2 Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation
R2 Score: 0.02 (+/- 0.02)


En este caso, el modelo que se va a utilizar es una regresión lineal con OLS

In [35]:
model = LinearRegression(fit_intercept=True)
#Se utiliza el cross validation con 100 iteraciones y se obtiene el score de cada iteración
kf = KFold(n_splits=50, shuffle=True, random_state=42)
#Se utiliza el score R2 para evaluar el modelo
scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
print("Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation") 
print("R2 Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation
R2 Score: 0.02 (+/- 0.01)


En este caso, el modelo que se va utilizar es la normalizacion de X

In [32]:
# Normalizar los datos
from sklearn.preprocessing import StandardScaler    
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
#normalizar los datos de y_train
y_train_scaled = (y_train - y_train.mean()) / y_train.std()

In [36]:
model = LinearRegression()
#Se utiliza el cross validation con 100 iteraciones y se obtiene el score de cada iteración
kf = KFold(n_splits=50, shuffle=True, random_state=42)
#Se utiliza el score R2 para evaluar el modelo
scores = cross_val_score(model, X_train_scaled, y_train_scaled, cv=kf, scoring='r2')
print("Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation") 
print("R2 Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Obtenemos el valor de la media y la desviación estándar de los scores obtenidos en cada iteración del cross validation
R2 Score: 0.02 (+/- 0.01)


## Asignación 2 &#x1F921;

Evalue su modelo final en el conjunto de test (X_test y y_test). Su práctica será evaluada acorde a las técnicas aplicadas, la estimación de $E_{out}$, el valor de $E_{test}$, y contra las métricas obtenidas por sus compañeros.